In [1]:
import pandas as pd
import geopandas as gpd
import fiona

pd.set_option('display.max_columns', None)

In [2]:
pwd

'/Users/mmainza/Desktop/projects/city-comparison/data/processing'

In [20]:
msaGeo = gpd.read_file('../../../data/spatial/msa/us_msa_centroids.geojson')
msa = pd.read_excel('../../../data/geographies/cbsa.xlsx', converters={'FIPS State':str, 'CBSA Code':str})

In [21]:
msa = msa[['CBSA Code','CBSA Title','FIPS State']]
msa.drop_duplicates(subset='CBSA Code', inplace=True)

In [22]:
msaGeo = msaGeo[['GEOID','NAME','geometry']]

In [23]:
data = pd.merge(msaGeo,msa,how='left', left_on='GEOID', right_on='CBSA Code')

In [24]:
msaGeo.shape

(926, 3)

In [25]:
data.shape

(926, 6)

In [26]:
data.head()

,GEOID,NAME,geometry,CBSA Code,CBSA Title,FIPS State
0,12020,"Athens-Clarke County, GA",POINT (-83.21379 33.94901),12020,"Athens-Clarke County, GA",13
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA",POINT (-84.39957 33.69277),12060,"Atlanta-Sandy Springs-Alpharetta, GA",13
2,12100,"Atlantic City-Hammonton, NJ",POINT (-74.63373 39.46883),12100,"Atlantic City-Hammonton, NJ",34
3,12120,"Atmore, AL",POINT (-87.16162 31.12612),12120,"Atmore, AL",01
4,12140,"Auburn, IN",POINT (-84.99908 41.39758),12140,"Auburn, IN",18


In [27]:
data['geoid'] = data['FIPS State']+data['CBSA Code']

In [29]:
data.rename(columns={'CBSA Code':'cbsa','NAME':'area'}, inplace=True)

In [31]:
data = data[['geoid','cbsa','area','geometry']]

In [37]:
crs = {'init': 'epsg:4326'}
data = gpd.GeoDataFrame(data, crs=crs, geometry=data['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [39]:
data.to_file('../../../data/spatial/msa/usMsaCentroids.geojson', driver="GeoJSON")